In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
# things we need for Tensorflow
import numpy as np
import keras
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

/home/shubham/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/home/shubham/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['Which mopeds do you have?',
    'What kinds of mopeds are there?',
    'What do you rent?'],
   'responses': ['We rent Yamaha, Piaggio and Vespa mopeds',
    'We have Piaggio, Vespa and Yamaha mopeds'

In [5]:
X = []
y_text = []

In [6]:
for doc in intents['intents']:
    print(doc['tag'])
    for p in doc['patterns']:
        # print(p)
        # print(" ".join([stemmer.stem(i) for i in nltk.word_tokenize((p.lower()))]))
        X.append(" ".join([stemmer.stem(i) for i in nltk.word_tokenize((p.lower()))]))
        y_text.append(doc['tag'])

greeting
goodbye
thanks
hours
mopeds
payments
opentoday
rental


In [7]:
X

['hi',
 'how ar you',
 'is anyon ther ?',
 'hello',
 'good day',
 'bye',
 'see you lat',
 'goodby',
 'thank',
 'thank you',
 "that 's help",
 'what hour ar you op ?',
 'what ar yo hour ?',
 'when ar you op ?',
 'which mop do you hav ?',
 'what kind of mop ar ther ?',
 'what do you rent ?',
 'do you tak credit card ?',
 'do you acceiv mastercard ?',
 'ar you cash on ?',
 'ar you op today ?',
 'when do you op today ?',
 'what ar yo hour today ?',
 'can we rent a mop ?',
 "i 'd lik to rent a mop",
 'how doe thi work ?']

In [8]:
y_text

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'hours',
 'hours',
 'hours',
 'mopeds',
 'mopeds',
 'mopeds',
 'payments',
 'payments',
 'payments',
 'opentoday',
 'opentoday',
 'opentoday',
 'rental',
 'rental',
 'rental']

In [9]:
len(X)

26

# Convert text into numbers

In [10]:
cv = CountVectorizer()

In [11]:
X_cv = cv.fit_transform(X)

In [12]:
X_cv = X_cv.toarray()

In [13]:
type(X_cv)

numpy.ndarray

In [14]:
X_cv.shape

(26, 44)

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
label_encoder_y = LabelEncoder()

In [17]:
y = label_encoder_y.fit_transform(y_text)

In [18]:
y

array([1, 1, 1, 1, 1, 0, 0, 0, 7, 7, 7, 2, 2, 2, 3, 3, 3, 5, 5, 5, 4, 4,
       4, 6, 6, 6])

In [19]:
y.shape

(26,)

In [20]:
y_mapping = {}
for i in range(len(y_text)):
    if y_text[i] not in y_mapping:
        y_mapping[y[i]] = y_text[i]

print(y_mapping)

{1: 'greeting', 0: 'goodbye', 7: 'thanks', 2: 'hours', 3: 'mopeds', 5: 'payments', 4: 'opentoday', 6: 'rental'}


In [21]:
y_text

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'hours',
 'hours',
 'hours',
 'mopeds',
 'mopeds',
 'mopeds',
 'payments',
 'payments',
 'payments',
 'opentoday',
 'opentoday',
 'opentoday',
 'rental',
 'rental',
 'rental']

In [22]:
y

array([1, 1, 1, 1, 1, 0, 0, 0, 7, 7, 7, 2, 2, 2, 3, 3, 3, 5, 5, 5, 4, 4,
       4, 6, 6, 6])

# Train model

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
random_forest = RandomForestClassifier()

In [25]:
random_forest.fit(X_cv, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

# Testing

In [26]:
test_data = "How are you"
test_data = " ".join([stemmer.stem(i) for i in nltk.word_tokenize((test_data.lower()))])

In [27]:
X_cv_test = cv.transform([test_data]).toarray()

In [28]:
X_cv_test

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [29]:
result = random_forest.predict(X_cv_test)

In [30]:
print(type(result))
int(result)

<class 'numpy.ndarray'>


1

In [31]:
predicted_tag = y_text[int(result)]
print(predicted_tag)

greeting


In [33]:
import random
for doc in intents['intents']:
    if doc['tag'] == predicted_tag:
        #print(doc['responses'])
        print(random.choice(doc['responses']))
    

Good to see you again


In [35]:
#for doc in intents['intents']:
#    print(doc['responses'])